In [58]:
import pandas as pd
from pandas import Series
import numpy as np
from functools import reduce
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense
from keras.initializers import glorot_normal
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy import array
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support

import keras
from scipy.special import softmax

In [2]:
data = pd.read_csv("data/orig/cleaned_data_notEncoded.csv")

In [3]:
data['label_away'] = data['label_home'].apply(lambda x: -1 * x)

data['home_won'] = data['label_home'].apply(lambda x: 1 if x == 1 else 0)
data['home_loss'] = data['label_home'].apply(lambda x: 1 if x == -1 else 0)
data['home_draw'] = data['label_home'].apply(lambda x: 1 if x == 0 else 0)

data['away_won'] = data['label_away'].apply(lambda x: 1 if x == 1 else 0)
data['away_loss'] = data['label_away'].apply(lambda x: 1 if x == -1 else 0)
data['away_draw'] = data['label_away'].apply(lambda x: 1 if x == 0 else 0)

In [4]:
match_data = pd.read_csv("data/orig/match_data.csv")
match_data.columns.values

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'aw

In [5]:
relevant_match_data = match_data[['season', 'match_api_id', 'date', 'home_team_api_id', 'away_team_api_id']]

In [6]:
data_with_season_date = pd.merge(data, relevant_match_data, on='match_api_id')

In [7]:
data_with_season_date.columns.values

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'home_buildUpPlaySpeed',
       'home_buildUpPlayDribbling', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationCrossing',
       'home_chanceCreationShooting', 'home_defencePressure',
       'home_defenceAggression', 'home_defenceTeamWidth',
       'home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_preferred_foot',
       'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_

In [8]:
grouped_home = data_with_season_date.groupby(['season', 'home_team_api_id'])

In [9]:
names = []
groups = []
for name, group in grouped_home:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)

In [10]:
print(names[-2])
groups[-2]

('2015/2016', 208931)


,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,...,home_won,home_loss,home_draw,away_won,away_loss,away_draw,season,date,home_team_api_id,away_team_api_id
11369,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-11-01 00:00:00,208931,9876
11388,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,...,0,1,0,1,0,0,2015/2016,2015-11-22 00:00:00,208931,8533
11406,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-12-06 00:00:00,208931,8564
11424,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-12-20 00:00:00,208931,9885
11443,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,1,0,0,0,1,0,2015/2016,2016-01-09 00:00:00,208931,8600
11452,1.0,3.0,5.0,7.0,1.0,3.0,5.0,7.0,9.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-08-30 00:00:00,208931,8636
11505,1.0,3.0,5.0,7.0,1.0,3.0,5.0,7.0,9.0,4.0,...,1,0,0,0,1,0,2015/2016,2016-02-12 00:00:00,208931,8686
11536,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,...,1,0,0,0,1,0,2015/2016,2016-03-13 00:00:00,208931,9891
11635,1.0,3.0,5.0,7.0,1.0,3.0,5.0,7.0,9.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-09-20 00:00:00,208931,8535
11641,1.0,3.0,5.0,7.0,1.0,3.0,5.0,7.0,9.0,5.0,...,1,0,0,0,1,0,2015/2016,2015-09-23 00:00:00,208931,9875


In [11]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['home_win_record'] = df['home_won'].cumsum()
    df['home_loss_record'] = df['home_loss'].cumsum()
    df['home_draw_record'] = df['home_draw'].cumsum()
    df['season'] = name[0]
    df['home_team_api_id'] = name[1]
    new_groups += [df]

In [12]:
new_groups[4]

,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,...,away_won,away_loss,away_draw,season,date,home_team_api_id,away_team_api_id,home_win_record,home_loss_record,home_draw_record
6674,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,0,0,1,2008/2009,2008-08-16 00:00:00,8178,9789,0,0,1
6687,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,0,0,1,2008/2009,2008-10-31 00:00:00,8178,8721,0,0,2
6697,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,1,0,0,2008/2009,2008-11-15 00:00:00,8178,10189,0,1,2
6709,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,1,0,0,2008/2009,2008-11-29 00:00:00,8178,9823,0,2,2
6723,1.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,5.0,4.0,...,1,0,0,2008/2009,2008-12-13 00:00:00,8178,8398,0,3,2
6740,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,1,0,0,2008/2009,2009-02-07 00:00:00,8178,10269,0,4,2
6761,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,1,0,0,2008/2009,2009-02-22 00:00:00,8178,9790,0,5,2
6776,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,1,0,2008/2009,2009-03-08 00:00:00,8178,9911,1,5,2
6825,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,0,0,1,2008/2009,2009-04-25 00:00:00,8178,8295,1,5,3
6832,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,...,0,0,1,2008/2009,2008-08-30 00:00:00,8178,8226,1,5,4


In [13]:
df_home_stats = pd.concat(new_groups)

In [14]:
df_home_stats.shape

(20060, 914)

In [15]:
df_home_stats.columns.values

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'home_buildUpPlaySpeed',
       'home_buildUpPlayDribbling', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationCrossing',
       'home_chanceCreationShooting', 'home_defencePressure',
       'home_defenceAggression', 'home_defenceTeamWidth',
       'home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_preferred_foot',
       'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_

In [16]:
grouped_away = df_home_stats.groupby(['season', 'away_team_api_id'])

In [17]:
names = []
groups = []
for name, group in grouped_away:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)
print(names[4])
groups[1]

('2008/2009', 8226)


,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,...,away_won,away_loss,away_draw,season,date,home_team_api_id,away_team_api_id,home_win_record,home_loss_record,home_draw_record
14975,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,1,0,2008/2009,2008-11-01 00:00:00,8548,8066,1,0,0
14995,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,0,1,2008/2009,2008-11-22 00:00:00,8597,8066,2,0,2
15077,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,0,1,2008/2009,2009-05-16 00:00:00,8597,8066,5,2,5
14997,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,1,0,2008/2009,2008-11-29 00:00:00,9925,8066,4,1,0
15074,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,1,0,2008/2009,2009-05-13 00:00:00,9927,8066,2,5,3
15099,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,1,0,2008/2009,2008-10-04 00:00:00,9938,8066,6,6,4
14982,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,...,0,0,1,2008/2009,2008-11-08 00:00:00,10251,8066,0,0,1


In [18]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['away_win_record'] = df['away_won'].cumsum()
    df['away_loss_record'] = df['away_loss'].cumsum()
    df['away_draw_record'] = df['away_draw'].cumsum()
    df['season'] = name[0]
    df['away_team_api_id'] = name[1]
    new_groups += [df]

In [19]:
df_final = pd.concat(new_groups)

In [20]:
df_final.shape

(20060, 917)

In [21]:
df_final.columns.values

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'home_buildUpPlaySpeed',
       'home_buildUpPlayDribbling', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationCrossing',
       'home_chanceCreationShooting', 'home_defencePressure',
       'home_defenceAggression', 'home_defenceTeamWidth',
       'home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_preferred_foot',
       'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_

In [22]:
df_final['season'].values

array(['2008/2009', '2008/2009', '2008/2009', ..., '2015/2016',
       '2015/2016', '2015/2016'], dtype=object)

In [23]:
df_final['season'] = pd.factorize(df_final['season'])[0]

In [24]:
df_final = df_final.drop(columns=['label_away', 'home_won', 'home_loss', 'home_draw',
                                  'away_won', 'away_loss', 'away_draw', 'date'])

In [25]:
desired_columns = ['season', 'home_win_record', 'home_loss_record', 'home_draw_record',
                  'away_win_record', 'away_loss_record', 'away_draw_record']

for i in range(1, 12):
    for status in ['home_player', 'away_player']:
        posX_name = status + '_X' + str(i)
        posY_name = status + '_Y' + str(i)
        
        rating_name = status + '_' + str(i) + '_' + 'overall_rating'
        desired_columns.extend([posX_name, posY_name, rating_name])

df_rel = df_final[desired_columns]
df_rel.columns.values

array(['season', 'home_win_record', 'home_loss_record',
       'home_draw_record', 'away_win_record', 'away_loss_record',
       'away_draw_record', 'home_player_X1', 'home_player_Y1',
       'home_player_1_overall_rating', 'away_player_X1', 'away_player_Y1',
       'away_player_1_overall_rating', 'home_player_X2', 'home_player_Y2',
       'home_player_2_overall_rating', 'away_player_X2', 'away_player_Y2',
       'away_player_2_overall_rating', 'home_player_X3', 'home_player_Y3',
       'home_player_3_overall_rating', 'away_player_X3', 'away_player_Y3',
       'away_player_3_overall_rating', 'home_player_X4', 'home_player_Y4',
       'home_player_4_overall_rating', 'away_player_X4', 'away_player_Y4',
       'away_player_4_overall_rating', 'home_player_X5', 'home_player_Y5',
       'home_player_5_overall_rating', 'away_player_X5', 'away_player_Y5',
       'away_player_5_overall_rating', 'home_player_X6', 'home_player_Y6',
       'home_player_6_overall_rating', 'away_player_X6', 'away_pl

In [26]:
df_rel.shape

(20060, 73)

In [27]:
df_label = df_final['label_home']
df_label.shape

(20060,)

In [28]:
X = df_rel.values
y = df_label.values

In [29]:
y = [i + 1 for i in y]
y = array(y)
y = to_categorical(y)
y.shape

(20060, 3)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(16048, 73)
(16048, 3)
(4012, 73)
(4012, 3)


In [43]:
def recall(y_true, y_pred):
    print(type(y_true))
    print(tf.keras.backend.get_value(y_true))
    recall_vals = recall_score(y_true, y_pred, average=None)
    recall_vals = tf.convert_to_tensor(recall_vals, dtype=tf.float32)
    return recall_vals

def createToyModel(input_dim):
    
    xavier_init = glorot_normal()
    
    model = Sequential()
    model.add(Dense(units=27, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=9, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=3, activation='relu', kernel_initializer=xavier_init))
    opt = Adam(lr=1e-4)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [44]:
model = createToyModel(X_train.shape[1])

In [45]:
model.fit(x=X_train, y=y_train, batch_size=8, shuffle=True, epochs=15, 
         validation_data=(X_test, y_test))

Train on 16048 samples, validate on 4012 samples
Epoch 1/15
16048/16048 [==============================] - 2s 107us/step - loss: 1.4821 - acc: 0.3410 - val_loss: 1.0714 - val_acc: 0.4589
Epoch 2/15
16048/16048 [==============================] - 2s 95us/step - loss: 1.0648 - acc: 0.4579 - val_loss: 1.0576 - val_acc: 0.4586
Epoch 3/15
16048/16048 [==============================] - 2s 96us/step - loss: 1.0581 - acc: 0.4579 - val_loss: 1.0552 - val_acc: 0.4609
Epoch 4/15
16048/16048 [==============================] - 2s 96us/step - loss: 1.0490 - acc: 0.4602 - val_loss: 1.0455 - val_acc: 0.4594
Epoch 5/15
16048/16048 [==============================] - 2s 97us/step - loss: 1.0361 - acc: 0.4673 - val_loss: 1.0271 - val_acc: 0.4673
Epoch 6/15
16048/16048 [==============================] - 2s 98us/step - loss: 1.0299 - acc: 0.4809 - val_loss: 1.0267 - val_acc: 0.4843
Epoch 7/15
16048/16048 [==============================] - 2s 96us/step - loss: 1.0247 - acc: 0.4821 - val_loss: 1.0291 - val_acc

In [48]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [61]:
y_pred = model.predict(x=X_test)

p_score_avg, r_score_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(y_pred, axis=1).astype(int), 3), average='weighted')
print("Averages")
print(p_score_avg)
print(r_score_avg)
print(f1_score_avg)


p_scores, r_scores, f1_scores, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(y_pred, axis=1).astype(int), 3), average=None)
print("Per Class")
print(p_scores)
print(r_scores)
print(f1_scores)

Averages
0.5575679227008141
0.5077268195413759
0.4194703438828939
Per Class
[0.58659218 0.640625   0.49281736]
[0.27851459 0.03942308 0.91309071]
[0.37769784 0.07427536 0.64013709]
